In [ ]:
import requests
import pandas as pd
import sys
import json

# Getting temperature from 2024:

# Load API key from file
with open(r"C:\Users\joice\OneDrive\Documentos\Ironhack\ML course\secrets\wheater_api.txt", "r") as file:
    api_key = file.read().strip() 

# Make the request using the loaded key
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Barcelona/2024-01-01/2024-12-20?unitGroup=metric&include=days&key={api_key}&contentType=json"

# API Request
response = requests.request(
    "GET",
    url
)

# Check for errors
if response.status_code != 200:
    print('Unexpected Status code: ', response.status_code)
    sys.exit()

# Parse the JSON response
jsonData = response.json()

# Extract the 'days' key from the response
days = jsonData.get('days', [])

# Convert the data to a Pandas DataFrame
weather = pd.DataFrame(days)

# Display the first few rows of the DataFrame
print(weather.head())


     datetime  datetimeEpoch  tempmax  tempmin  temp  feelslikemax  \
0  2024-01-01     1704063600     13.6      5.1   8.1          13.6   
1  2024-01-02     1704150000     14.5      3.7   9.8          14.5   
2  2024-01-03     1704236400     17.2     12.2  14.5          17.2   
3  2024-01-04     1704322800     15.7      7.7  11.3          15.7   
4  2024-01-05     1704409200     13.6      8.4  10.8          13.6   

   feelslikemin  feelslike  dew  humidity  ...  sunriseEpoch    sunset  \
0           3.4        7.0  3.7      74.2  ...    1704093452  17:32:00   
1           1.9        9.0  1.8      59.0  ...    1704179858  17:32:50   
2          12.2       14.5  7.9      64.8  ...    1704266262  17:33:43   
3           5.6       10.8  8.3      82.4  ...    1704352664  17:34:37   
4           6.6       10.4  7.6      81.3  ...    1704439063  17:35:33   

   sunsetEpoch moonphase              conditions  \
0   1704126720      0.68        Partially cloudy   
1   1704213170      0.71      

In [14]:
weather.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source,tzoffset
0,2024-01-01,1704063600,13.6,5.1,8.1,13.6,3.4,7.0,3.7,74.2,...,1704093452,17:32:00,1704126720,0.68,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,"[LEBL, D1298, LEGE, 08184099999, LELL, 0818109...",obs,NaN
1,2024-01-02,1704150000,14.5,3.7,9.8,14.5,1.9,9.0,1.8,59.0,...,1704179858,17:32:50,1704213170,0.71,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[LEBL, D1298, 08184099999, LELL, 08181099999, ...",obs,NaN
2,2024-01-03,1704236400,17.2,12.2,14.5,17.2,12.2,14.5,7.9,64.8,...,1704266262,17:33:43,1704299623,0.74,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[LEBL, D1298, 08184099999, LELL, 08181099999, ...",obs,NaN
3,2024-01-04,1704322800,15.7,7.7,11.3,15.7,5.6,10.8,8.3,82.4,...,1704352664,17:34:37,1704386077,0.75,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[LEBL, D1298, 08184099999, LELL, 08181099999, ...",obs,NaN
4,2024-01-05,1704409200,13.6,8.4,10.8,13.6,6.6,10.4,7.6,81.3,...,1704439063,17:35:33,1704472533,0.80,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"[LEBL, D1298, 08184099999, LELL, 08181099999, ...",obs,NaN


In [15]:
import requests
import pandas as pd

# Define the function to fetch data
def fetch_data_from_api(resource_id, batch_size=1000):
    """
    Fetches data from the Barcelona Open Data API.
    
    Args:
        resource_id (str): The resource ID of the dataset.
        batch_size (int): The number of rows to fetch per batch (default is 1000).
    
    Returns:
        pd.DataFrame: A DataFrame containing the fetched data.
    """
    url = "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search"
    offset = 0
    all_records = []
    
    while True:
        params = {
            "resource_id": resource_id,
            "limit": batch_size,
            "offset": offset
        }
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Failed to fetch data for resource_id {resource_id}. HTTP Status code: {response.status_code}")
            break
        
        data = response.json()
        records = data.get("result", {}).get("records", [])
        
        if not records:
            break
        
        all_records.extend(records)
        offset += batch_size
    
    return pd.DataFrame(all_records)

# Dictionary containing dataset names and resource IDs
datasets = {
    "weather old": "0e3b6840-7dff-4731-a556-44fac28a7873",
    "energy 2024": "6f3a256a-dba6-46ac-88a4-546b93cb46da",
    "energy 2023": "25caee3a-a9f2-4e1a-8c8b-1b5fcac4f152",
    "energy 2022": "e685fa1a-9fbe-41a4-98a0-173464cd0c5f",
    "population 2024": "fc597601-a291-4811-ad02-c58e32784692",
    "population 2023": "f70020ae-c41a-438d-8983-a372628c197b",
    "population 2022": "78b965d3-b2dc-4f23-91b9-59caa45bc334",
    "house_size 2024": "c21adc95-814c-4b6e-a642-f5c479399992",
    "house_size 2023": "7a95887a-dbd3-421a-af26-b3fa743baadf",
    "house_size 2022": "08fbac59-9d1c-409b-ad76-44231b1dccd6",
    "house_age 2024": "e6e1be4d-0dd0-45b4-9458-43f55751afb4",
    "house_age 2023": "c612aed5-4ab0-4abb-8afe-2d64a5eef30c",
    "house_age 2022": "1f23cb2e-dc8f-4c82-b5a6-c52418a9f1db"
}

# Fetch all datasets using the function and store them in a dictionary
dataframes = {}
for name, resource_id in datasets.items():
    print(f"Fetching data for {name}...")
    dataframes[name] = fetch_data_from_api(resource_id)

# Single dataset with 2023 and previous temperature averages 
weather_old = dataframes["weather old"]


# Union datasets by type
energy = pd.concat(
    [
        dataframes["energy 2024"],
        dataframes["energy 2023"],
        dataframes["energy 2022"]
    ],
    ignore_index=True
)

population = pd.concat(
    [
        dataframes["population 2024"],
        dataframes["population 2023"],
        dataframes["population 2022"]
    ],
    ignore_index=True
)

house_size = pd.concat(
    [
        dataframes["house_size 2024"],
        dataframes["house_size 2023"],
        dataframes["house_size 2022"]
    ],
    ignore_index=True
)

house_age = pd.concat(
    [
        dataframes["house_age 2024"],
        dataframes["house_age 2023"],
        dataframes["house_age 2022"]
    ],
    ignore_index=True
)

Fetching data for weather old...
Fetching data for energy 2024...
Fetching data for energy 2023...
Fetching data for energy 2022...
Fetching data for population 2024...
Fetching data for population 2023...
Fetching data for population 2022...
Fetching data for house_size 2024...
Fetching data for house_size 2023...
Fetching data for house_size 2022...
Fetching data for house_age 2024...
Fetching data for house_age 2023...
Fetching data for house_age 2022...


In [17]:
house_size.head()

,Codi_districte,Nom_districte,Nom_barri,Seccio_censal,Sup_mitjana_m2,Codi_barri,_id,Any
0,1,Ciutat Vella,el Raval,1,76.7,1,1,2024
1,1,Ciutat Vella,el Raval,2,67.9,1,2,2024
2,1,Ciutat Vella,el Raval,3,59.1,1,3,2024
3,1,Ciutat Vella,el Raval,4,65,1,4,2024
4,1,Ciutat Vella,el Raval,5,77.8,1,5,2024


In [18]:
house_size['Any'].value_counts()

Any
2024    1068
2023    1068
2022    1068
Name: count, dtype: int64

In [19]:
house_size['Nom_barri'].unique()

array(['el Raval', 'el Barri Gòtic', 'la Barceloneta',
       'Sant Pere, Santa Caterina i la Ribera', 'el Fort Pienc',
       'la Sagrada Família', "la Dreta de l'Eixample",
       "l'Antiga Esquerra de l'Eixample",
       "la Nova Esquerra de l'Eixample", 'Sant Antoni', 'el Poble Sec',
       'la Marina del Prat Vermell', 'la Marina de Port',
       'la Font de la Guatlla', 'Hostafrancs', 'la Bordeta',
       'Sants-Badal', 'Sants', 'les Corts', 'la Maternitat i Sant Ramon',
       'Pedralbes', 'Vallvidrera, el Tibidabo i les Planes', 'Sarrià',
       'les Tres Torres', 'Sant Gervasi- la Bonanova',
       'la Vila de Gràcia', 'Sant Gervasi- Galvany',
       'el Putxet i el Farró', 'Vallcarca i els Penitents', 'el Coll',
       'la Salut', "el Camp d'en Grassot i Gràcia Nova",
       'el Baix Guinardó', 'Can Baró', 'el Guinardó',
       "la Font d'en Fargues", 'el Carmel', 'la Teixonera',
       'Sant Genís dels Agudells', 'Montbau', "la Vall d'Hebron",
       'la Clota', 'Horta', 'Vi

In [20]:
energy[energy['Any'] == '2024'].tail()

,Sector_Economic,Tram_Horari,Codi_Postal,Valor,_id,Data,Any
156520,Serveis,De 00:00:00 a 05:59:59 h,8042,5402,156521,2024-10-31T00:00:00,2024
156521,Serveis,De 06:00:00 a 11:59:59 h,8042,8751,156522,2024-10-31T00:00:00,2024
156522,Serveis,De 12:00:00 a 17:59:59 h,8042,10548,156523,2024-10-31T00:00:00,2024
156523,Serveis,De 18:00:00 a 23:59:59 h,8042,10671,156524,2024-10-31T00:00:00,2024
156524,Serveis,No consta,8042,552,156525,2024-10-31T00:00:00,2024


In [21]:
energy['Any'].value_counts()

Any
2022    251070
2023    241125
2024    156525
Name: count, dtype: int64

In [23]:
energy.tail()

,Sector_Economic,Tram_Horari,Codi_Postal,Valor,_id,Data,Any
648715,Serveis,De 00:00:00 a 05:59:59 h,8042,24391,251066,2022-12-31T00:00:00,2022
648716,Serveis,De 06:00:00 a 11:59:59 h,8042,30408,251067,2022-12-31T00:00:00,2022
648717,Serveis,De 12:00:00 a 17:59:59 h,8042,32158,251068,2022-12-31T00:00:00,2022
648718,Serveis,De 18:00:00 a 23:59:59 h,8042,32510,251069,2022-12-31T00:00:00,2022
648719,Serveis,No consta,8042,0,251070,2022-12-31T00:00:00,2022


In [24]:
energy['Codi_Postal'].unique()

array(['8001', '8002', '8003', '8005', '8004', '8006', '8007', '8008',
       '8009', '8010', '8011', '8012', '8013', '8014', '8015', '8016',
       '8017', '8018', '8019', '8020', '8021', '8022', '8023', '8024',
       '8025', '8026', '8027', '8028', '8029', '8030', '8031', '8032',
       '8033', '8034', '8035', '8036', '8037', '8038', '8039', '8040',
       '8041', '8042'], dtype=object)

In [25]:
energy['Codi_Postal'].value_counts()

Codi_Postal
8005    17600
8018    17600
8028    17595
8029    17595
8021    17595
8006    17595
8007    17590
8034    16995
8008    16875
8017    16795
8015    16620
8025    16320
8014    15585
8013    15330
8036    14895
8012    14735
8011    14735
8024    14730
8010    14725
8030    14715
8020    14715
8038    14715
8002    14710
8019    14710
8026    14710
8016    14710
8001    14710
8040    14705
8022    14705
8041    14695
8027    14685
8004    14655
8003    14645
8031    14620
8032    14620
8033    14620
8023    14620
8035    14620
8009    14620
8037    14620
8042    14620
8039    14465
Name: count, dtype: int64

In [49]:
pd.set_option('display.max_rows', None)
house_size[['Nom_barri', 'Codi_barri']].drop_duplicates()

,Nom_barri,Codi_barri
0,el Raval,1
21,el Barri Gòtic,2
30,la Barceloneta,3
41,"Sant Pere, Santa Caterina i la Ribera",4
54,el Fort Pienc,5
74,la Sagrada Família,6
108,la Dreta de l'Eixample,7
137,l'Antiga Esquerra de l'Eixample,8
163,la Nova Esquerra de l'Eixample,9
203,Sant Antoni,10


In [26]:
pd.set_option('display.max_rows', None)
house_age[['Nom_barri', 'Codi_barri']].drop_duplicates()

,Nom_barri,Codi_barri
0,el Raval,1
21,el Barri Gòtic,2
30,la Barceloneta,3
41,"Sant Pere, Santa Caterina i la Ribera",4
54,el Fort Pienc,5
74,la Sagrada Família,6
108,la Dreta de l'Eixample,7
137,l'Antiga Esquerra de l'Eixample,8
163,la Nova Esquerra de l'Eixample,9
203,Sant Antoni,10


In [27]:
pd.set_option('display.max_rows', None)
population[['Nom_Barri', 'Codi_Barri']].drop_duplicates()

,Nom_Barri,Codi_Barri
0,el Raval,1
21,el Barri Gòtic,2
30,la Barceloneta,3
41,"Sant Pere, Santa Caterina i la Ribera",4
54,el Fort Pienc,5
74,la Sagrada Família,6
108,la Dreta de l'Eixample,7
137,l'Antiga Esquerra de l'Eixample,8
163,la Nova Esquerra de l'Eixample,9
203,Sant Antoni,10


In [29]:
weather_old.head()

,Temp_Mitjana_Agost,Temp_Mitjana_Octubre,Temp_Mitjana_Febrer,Temp_Mitjana_Juny,Temp_Mitjana_Novembre,Temp_Mitjana_Juliol,Temp_Mitjana_Setembre,Temp_Mitjana_Desembre,Temp_Mitjana_Gener,Temp_Mitjana_Abril,Temp_Mitjana_Marc,_id,Any,Temp_Mitjana_Maig
0,22,16.3,7.3,19.1,9.2,21.4,20.3,7.3,6.7,11.4,11.6,1,1780,16.3
1,22.6,15,8.9,18,11.8,22,19.1,10.3,7.5,13.7,10.9,2,1781,16.8
2,22.6,13.8,6.1,20.1,7.4,22.3,19.9,7.5,8.2,10.5,9.8,3,1782,15.4
3,21.4,15.5,8.6,19.1,11,22.9,18.6,10.4,8.3,12.4,9.6,4,1783,15
4,20.5,13.8,7.1,20.7,8.5,22,20,5.3,7.3,10.7,10.8,5,1784,16.7


In [32]:
import pickle

# Combine all raw DataFrames into a dictionary
raw_data = {
    "weather": weather,
    "weather_old": weather_old,
    "energy": energy,
    "population": population,
    "house_size": house_size,
    "house_age": house_age
}

# Save the dictionary to a pickle file
with open("raw_data.pkl", "wb") as f:
    pickle.dump(raw_data, f)

print("All raw DataFrames have been pickled into 'raw_data.pkl'.")

All raw DataFrames have been pickled into 'raw_data.pkl'.
